In [1]:
import sys


sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))
import os

os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid.services.store_registry import StoreRegistry
from sem_covid import config
from sem_covid.adapters.data_source import BinaryDataSource
from sem_covid.services.ml_pipelines.pwdb_classifiers_pipeline import FeatureEngineering, EMBEDDING_COLUMN, TEXTUAL_DATA

In [2]:
import tensorflow_hub as hub

In [250]:
from sklearn.metrics.pairwise import pairwise_distances
from sem_covid import config
from sem_covid.services.store_registry import StoreRegistry
from sem_covid.services.data_registry import Dataset

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean, pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt
import plotly.express as px
import concurrent.futures

In [83]:
from transformers import AutoTokenizer, AutoModel

In [84]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/bert-base-uncased-eurlex")
model = AutoModel.from_pretrained("nlpaueb/bert-base-uncased-eurlex")

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/bert-base-uncased-eurlex were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [113]:
inputs = tokenizer('Hello', return_tensors='pt')

In [115]:
output = model(**inputs)

In [105]:
import numpy as np

In [205]:
from scipy.spatial.distance import cosine, euclidean

In [153]:
def cosine_sim(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


In [172]:
def encode_text(text: str, model, tokenizer):
    return model(**tokenizer(text, return_tensors='pt'))['pooler_output'].detach().numpy()[0]

In [228]:
h1 = encode_text('Tomorrow the judge will do the hearing', model, tokenizer)
h1[:5]

array([-0.1866121,  0.6182396,  0.9996494, -0.5832228, -0.9880023],
      dtype=float32)

In [229]:
h2 = encode_text('the court proceeding will place tomorrow', model, tokenizer)
h2[:5]

array([-0.16304274,  0.56621265,  0.9995914 , -0.50024855, -0.98523676],
      dtype=float32)

In [230]:
cosine_sim(h1, h2)

0.99654305

In [199]:
1- cosine(h1,h2)

0.9426586627960205

In [231]:
1/(1+euclidean(h1,h2))


0.38564461516762555

In [9]:
class FeatureEngineeringBERT(FeatureEngineering):

    def __init__(self, feature_store_name: str):
        super().__init__(feature_store_name)
        self.model = None

    def load_language_model(self):
        model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
        self.model = hub.load(model_url)

    def compute_document_embeddings(self):
        assert self.model is not None
        #self.prepared_dataset[DOCUMENT_EMBEDDING] = self.model(
        #    self.prepared_dataset[DOCUMENT_TEXT].tolist()).numpy().tolist()
        self.df[EMBEDDING_COLUMN] = self.df[TEXTUAL_DATA].apply(lambda x: self.model([x]).numpy().tolist())

In [10]:
tmp_pipeline = FeatureEngineeringBERT(feature_store_name='fs_pwdb_bert')

In [11]:
tmp_pipeline.execute()

InvalidArgumentError:  input must be a vector, got shape: []
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/text_preprocessor/tokenize/StringSplit/StringSplit}}]] [Op:__inference_restored_function_body_12953]

Function call stack:
restored_function_body
